In [1]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np
import pickle
import os
import glob

In [2]:
# G = nx.read_edgelist("./IsA_Graph2.edgelist", delimiter=" ", create_using=nx.DiGraph)
G = nx.read_edgelist("ru_isa.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [3]:
l = Leafer(G)
# iterator = l.leafs_generator()

In [4]:
train, test = l.split_train_test(
    generation_depth=1,  # до какого уровня в топ. сортировке идти
    p=0.05,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.6,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.5,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.25, 0.3, 0.25, 0.1, 0.1],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка
)

In [5]:
len(train), len(test)

(10361, 441)

In [6]:
ls = [elem for elem in test if elem["case"] == "only_leafs_divided"]
len(ls)

66

In [7]:
train_count = {}
for elem in train:
    if elem["case"] in train_count.keys():
        train_count[elem["case"]] += 1

    else:
        train_count[elem["case"]] = 1

test_count = {}
for elem in test:
    if elem["case"] in test_count.keys():
        test_count[elem["case"]] += 1

    else:
        test_count[elem["case"]] = 1

In [8]:
train_count, test_count

({'only_leafs_divided': 1233,
  'only_child_leaf': 2034,
  'leafs_and_no_leafs': 1371,
  'simple_triplet_grandparent': 1836,
  'simple_triplet_2parent': 2932,
  'only_leafs_all': 955},
 {'simple_triplet_grandparent': 92,
  'leafs_and_no_leafs': 55,
  'only_leafs_divided': 66,
  'only_child_leaf': 112,
  'only_leafs_all': 45,
  'simple_triplet_2parent': 71})

In [9]:
num_leaks = 0
for vertex in l.collector.test_verteces:
    if vertex in l.collector.train_verteces:
        num_leaks += 1

In [10]:
num_leaks

0

In [11]:
num_leaks = 0
for vertex in l.collector.train_verteces:
    if vertex in l.collector.test_verteces:
        num_leaks += 1

In [12]:
num_leaks

0

In [13]:
all_data = glob.glob(os.path.join(".", "*isa.edgelist"))
for path in tqdm(all_data):
    lang = path.replace("./", "").replace("_isa.edgelist", "")

    G = nx.read_edgelist(path, delimiter="\t", create_using=nx.DiGraph)
    l = Leafer(G)
    train, test = l.split_train_test(
        generation_depth=1,
        p=0.05,
        p_divide_leafs=0.5,
        min_to_test_rate=0.5,
        weights=[0.25, 0.3, 0.25, 0.1, 0.1],
    )

    name_train = "../babel_datasets/train_" + lang + "_babel.pickle"
    name_test = "../babel_datasets/test_" + lang + "_babel.pickle"

    with open(name_train, "wb") as handle:
        pickle.dump(train, handle)

    with open(name_test, "wb") as handle:
        pickle.dump(test, handle)

100%|██████████| 6/6 [00:11<00:00,  1.93s/it]


In [14]:
ls = [elem for elem in test if elem["case"] == "only_leafs_divided"]

In [15]:
ls

[{'children': ['графема.n.1'],
  'parents': 'написанных_символов.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['Фонограмма_(лингвистика).n.1']},
 {'children': ['многоколосник_фенхельный.n.1'],
  'parents': 'гигантский_иссоп.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['Agastache_mexicana.n.1', 'agastache_nepetoides.n.1']},
 {'children': ['аятолла.n.1'],
  'parents': 'религиозный_лидер.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['гуру.n.1']},
 {'children': ['Херман,_Джерри.n.1', 'Glen_Phillips.n.1'],
  'parents': 'поэт-песенник.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['Блэйз_Бэйли.n.1', 'Сигурдур_Тораринссон.n.1']},
 {'children': ['Южноафриканская_щитковая_кобра.n.1'],
  'parents': 'старый_змей_коралловый_мир.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['азиатские_змеи_коралл.n.1', 'rhynchoelaps_australis.n.1']},
 {'children'

In [17]:
def predict_child_with_parent_and_grandparent(elem):
    """
    hyperhypenym: arthropod.n.01,
    hypernym: insect.n.01, hyponyms:
    (blackly)

    hyperhypenym: elem['grandparents'],
    hypernym: elem['parents'], hyponyms:
    elem['children']

    Fly is a hyponym for the word “insect".
    Predict hyponyms for the word “fly”. Answer:
    """

    # transformed_term = (
    #     "hyperhypenym: "
    #     + ", ".join(elem["grandparents"])
    #     + ", hypernym: "
    #     + elem["parents"]
    #     + ", hyponyms:"
    # )
    transformed_term = (
        ", ".join(elem["grandparents"])
        + " are hyponyms for the word '"
        + elem["parents"]
        + "'. Predict hyponyms for the word '"
        + elem["parents"]
        + "'. Answer:"
    )
    return transformed_term